In [50]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
from tqdm import tqdm
import time
import json
import re

class elastic:
    def __init__(self, INDEX_NAME, context_path="../data/wikipedia_documents.json"):
        self.index_name = INDEX_NAME
        try:
            self.es.transport.close()
        except:
            pass
        self.context_path = context_path
        config = {
            "host": "localhost", 
            "port": 9200,
            "timeout": 100,
            "max_retries": 10,
            "retry_on_timeout": True,
            }
        self.es = Elasticsearch([config])
        
        self.index_setting = {
            "settings": {
                "index": {
                    "analysis": {
                        "analyzer": {
                            "korean": {
                                "type": "custom",
                                "tokenizer": "nori_tokenizer",
                                "filter": ["shingle"],
                            }
                        }
                    }
                }
            },
            "mappings": {
                "properties": {
                    "text": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                    "title": {
                        "type": "text",
                        "analyzer": "korean",
                        "search_analyzer": "korean",
                    },
                }
            },
        }

    def build_elatic(self):
        with open(self.context_path) as file:
            json_data = json.load(file)
        docs = []
        for i, j in json_data.items():
            docs.append(
                {
                    "_index": "wikipedia",
                    "_source": {"text": j["text"], "title": j["title"]},
                }
            )

        if self.es.indices.exists(self.index_name):
            pass
        else:
            self.es.indices.create(index=self.index_name, body=self.index_setting)
            helpers.bulk(self.es, docs)

    def retrieve(self, query_or_dataset, topk):
        datas = []

        # changing the key into integer type
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if (
                "context" in query_or_dataset[i].keys()
                and "answers" in query_or_dataset[i].keys()
            ):
                cp["original_context"] = query_or_dataset[i]["context"]

            query = query_or_dataset[i]["question"]
            query = query.replace("/", "")
            query = query.replace("~", " ")
            res = self.es.search(index=self.index_name, q=query, size=topk+1)
            x = res["hits"]["hits"]
            context = []
            for docu in x:
                context.append(docu["_source"]["text"])
            cp["context"] = "///".join(context)
            datas.append(cp)

        return pd.DataFrame(datas)

    def retrieve_false(self, query_or_dataset, topk):
        datas = []
        for i in tqdm(range(len(query_or_dataset))):
            cp = {i: v for i, v in query_or_dataset[i].items()}
            if (
                "context" in query_or_dataset[i].keys()
                and "answers" in query_or_dataset[i].keys()
            ):
                cp["original_context"] = query_or_dataset[i]["context"]

            query = query_or_dataset[i]["question"]
            query = query.replace("/", "")
            query = query.replace("~", " ")
            res = self.es.search(index=self.index_name, q=query, size=topk+1)
            x = res["hits"]["hits"]
            context = []
            for docu in x:
                cont = docu["_source"]["text"]
                # groud truth 제거
                if cont == cp["original_context"]: continue
                context.append(docu["_source"]["text"])
            context = context[:topk]
            cp["context"] = "///".join(context)
            datas.append(cp)

        return pd.DataFrame(datas)

In [51]:
from datasets import load_from_disk

dataset = load_from_disk("../data/train_dataset")
train_datasets = dataset["train"]
valid_datasets = dataset["validation"]


In [52]:
x = elastic("wikipedia")
x.build_elatic()


<ipython-input-50-2385cbbed766>:67: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if self.es.indices.exists(self.index_name):
/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


## Train dataset retrieving for hard negatives

In [4]:
p = x.retrieve_false(train_datasets, 100)
p.to_csv("/opt/ml/data/train_elastic_top100_noanswer.csv")

100%|██████████| 3952/3952 [03:29<00:00, 18.84it/s]


## Validation dataset retrieving for hard negatives

In [6]:
val = x.retrieve_false(valid_datasets, 100)
val.to_csv("/opt/ml/data/valid_elastic_top100_noanswer.csv")

100%|██████████| 240/240 [00:12<00:00, 19.16it/s]


In [8]:
p.context[0][:1000]

'내각(內閣, cabinet)은 행정부의 주요 각료들로 구성되는 국가의 주요기관이다.\n\n의원내각제에서 내각은 수상과 여러 장관으로 조직되는 합의체로, 국가의 행정권을 담당하고 국회에 대한 연대책임을 갖는다. 의원내각제에 있어서 내각은 국가행정의 최고기관인 한편 국민이 구성시키는 의회에 의하여 철저히 견제되어 의회민주주의 체제를 이룬다.\n\n그 직접적 유래는 영국에서 국왕의 정치를 자문하던 추밀원에서 찾을 수 있다. 특히 내각은 추밀원의 일개 회의에서 시작하였다가 권한이 집중되어 분리된 기관으로, 이후 국왕의 실권이 사라지고 일명 웨스트민스터 시스템으로 불리는 의원내각제가 성립하면서 의회에 의한 민주적 행정부를 이루게 되었다.\n\n한편 국가원수에게 대부분의 권력이 집중되는 대통령중심제와 군주제에서 내각은 원칙적으로 의결권이 없거나 의결의 구속력이 없는 보좌기관에 불과한 경우가 많다.(예: 대한민국의 국무회의)\n\n대한민국은 국무회의가 내각에 속하며 권한이 대통령에 비해 제한적이다. 과거 왕조시대 때는 고려시대의 중서문하성, 중추원, 육부 또는 조선시대의 의정부와 육조가 내각과 비슷한 성향을 지니고 있었다.///이하의 국가들은 의회에서 행정부 수반을 선출한다는 점에서 내각제와 닮았다. 그리고 이들 국가들 중 상당수는 행정부 수반이 의회해산권을 갖고, 의회는 내각불신임권을 갖는데 이점 역시 내각제와 닮았다. 하지만 내각제와 달리 의회에서 선출되는 자는 행정부 수반의 지위 뿐만 아니라 국가 원수의 지위도 가진다. 그래서 의회에서 선출되는 자의 직위는 총리가 아니라, 대통령이다. 이처럼 1인이 행정부 수반과 국가 원수의 지위를 겸한다는 점에서 대통령중심제와 닮았다. 이상과 같은 이유로 이들 국가들의 정부 형태는 대통령제와 내각제의 절충형이지만, 행정 권한이 2인에게로 분리되어 있지 않으므로 이원집정부제는 아니다.\n\n \n* 나우루\n* 남아프리카 공화국\n* 마셜 제도\n* 마카오\n* 미얀마\n* 미크로네시아 연방\n* 보츠와나\n* 산마리노\n* 수리남

## Test dataset retrieval for elastic search

In [23]:
context_path = '../data/wikipedia_documents.json'
with open(context_path, 'r', encoding= "utf-8") as f:
    wiki = json.load(f)
search_corpus = list(dict.fromkeys([v['text'] for v in wiki.values()]))
search_corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [37]:
wiki["0"]

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [39]:
query_or_dataset = {int(k):v for k,v in wiki.items()}
query_or_dataset

dict

In [24]:
import pandas as pd
import numpy as np
import json

# load test dataset as dataframe
test = load_from_disk('../data/test_dataset')
test_datasets = test['validation']
test_datasets

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [49]:
test_datasets[0]

{'question': "유령'은 어느 행성에서 지구로 왔는가?", 'id': 'mrc-1-000653'}

In [53]:
test = x.retrieve(test_datasets, 100)
test.to_csv("/opt/ml/data/test_elastic_top100.csv")

 67%|██████▋   | 401/600 [00:20<00:08, 22.20it/s]